# Nesse caderno quero analisar as denuncias a empresas do Brasil, retiradas de uma planilha vazada ano passado.

# Quero a principio categorizar essas denuncias.

## Aluno: José Luan Silva do Nascimento

##### Tentei fazer o caderno para pegar reclamações do reclameaqui e apatir de uma nova reclamação inferir quando ela seria respondida, porém o site e aplicativo tem uma camada de segurança muito boa. Tentei pegar do site consumidor.gov, mas também não deu certo.

In [ ]:
import gdown
import pandas as pd

nome_arquivo = 'empresas.xlsx'
# Baixando o csv das denuncias
gdown.download(url='https://drive.google.com/file/d/1EbpEG7qB4nZkO8KzWFCe-_xa8EtJOFMe6TqIxwS7ByI/view?usp=sharing&format=xlsx', output=nome_arquivo, quiet=False, fuzzy=True)

df = pd.read_excel('empresas.xlsx', engine='openpyxl')
display(df)
json_output = df.to_json()


Downloading...
From: https://drive.google.com/uc?id=1a1XS9pA_CwHA0RnwtOrtOPCu0RA1kwgX
To: /content/empresas.tsv
100%|██████████| 5.11M/5.11M [00:00<00:00, 63.7MB/s]


'empresas.tsv'

In [ ]:
print(df.columns.tolist())
df['Denúncia'] = df['Denúncia'].apply(lambda x: x if isinstance(x, str) else None)
df = df.dropna(subset=['Denúncia'])
denuncias = df['Denúncia'].copy()

['ium', 'Carimbo de data/hora', 'Nome da empresa', 'Denúncia', 'CAPTCHA Matemático: 4 menos 4', 'Você é', 'Em quais categorias sua denúncia se encaixa?', 'Possui ÁUDIO, CAPTURAS DE TELA, VÍDEO, CONVERSAS COM COLEGAS?  Acesse exposedworkplaces.com/provas e nos envie ANONIMAMENTE', 'Qual nota você daria para esta empresa?', 'Possui provas? VÍDEO, AÚDIO, CAPTURAS DE TELA? acesse file.io e ', 'Possui provas? PRINT, ÁUDIO, VÍDEO... nos envie! Em caso de vídeos longos, poste no YouTube como NÃO-LISTADO e nos envie para anonimizá-lo.']


<ipython-input-82-b57452777c23>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Denúncia'] = df['Denúncia'].apply(lambda x: x if isinstance(x, str) else None)


In [ ]:
print(denuncias[0])

A Central de Segurança da 99 é um callcenter terceirizado, desses iguais os de cobrança ou pedido de doação, sabe? Pois é, os atendentes recebem um salário bizarro e uma cobrança desumana. Não esperem qualidade dos produtos e do atendimento


In [ ]:
import re
#Limpando os dados
def apenas_caracteres_importantes(texto) -> str:
   texto = re.sub(r'[^A-z\s\dáéíóúãõâêîôûàèìòùäëïöüçñ]+', '', texto, flags=re.DOTALL|re.IGNORECASE)
   texto = texto.replace('\t', ' ').replace('\n', ' ')
   texto = re.sub(r'\s+', ' ', texto, flags=re.DOTALL|re.IGNORECASE)
   texto = texto.strip()
   return texto


texto = ''' l.
Tive um burnout durante a minha estadia na empresa por conta de uma líder que abusava moralmente do time (com ameaças frequentes de demissão), e continua lá ainda depois de anos, ganhando prêmios de boa liderança
se vc for chamado pra trabalhar na área de UAU, simplesmente recuse a proposta... você está fazendo um livramento nessa vida, porque isso aqui é o inferno na terra, líderes E'''
print(apenas_caracteres_importantes(texto))
texto = "      Hello,     World!!!!!!,,,,     \n\n\n\n It is a beautiful , beautiful day!   \n garbage "
print(apenas_caracteres_importantes(texto))

l Tive um burnout durante a minha estadia na empresa por conta de uma líder que abusava moralmente do time com ameaças frequentes de demissão e continua lá ainda depois de anos ganhando prêmios de boa liderança se vc for chamado pra trabalhar na área de UAU simplesmente recuse a proposta você está fazendo um livramento nessa vida porque isso aqui é o inferno na terra líderes E
Hello World It is a beautiful beautiful day garbage


In [ ]:
print(denuncias)
print()
print(len(denuncias))

0        A Central de Segurança da 99 é um callcenter t...
1        a empresa fala de saúde mental, mas tem uma ta...
2        A liderança é tóxica. Inclusive tem um diretor...
3        Completamente delulus e absurdamente fora da r...
4        Contratam funcionários em sistema de terceiriz...
                               ...                        
15412    Os gerentes e coordenadores são pessoas podres...
15413     Empresa extremamente tóxica. Tem um CEO que s...
15414    Fui demitido e me "obrigaram" a mandar um PIX ...
15415    Jovens aprendizes sujeitos a se submeterem a t...
15416    A Empresa não assina carteira e já pagou aos f...
Name: Denúncia, Length: 15415, dtype: object

15415


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

pipeline_clean_text = Pipeline(
    [
        # Step 1: Deixa apenas os caracteres importantes para a analise
        ("apenas_caracteres_importantes", FunctionTransformer(apenas_caracteres_importantes))
    ]
)

denuncias_limpas = denuncias.apply(pipeline_clean_text.transform)
len(denuncias_limpas)

15415

In [ ]:
denuncias_limpas.str.split().apply(len).describe()

,Denúncia
count,15415.000000
mean,41.793578
std,46.689038
min,0.000000
25%,14.000000
50%,29.000000
75%,52.000000
max,683.000000


In [ ]:
import plotly.graph_objects as go

# Calculate word counts for each text in the cleaned training dataset
word_counts = denuncias_limpas.str.split().apply(len)

# Create a histogram plot using Plotly
fig = go.Figure(data=[go.Histogram(x=word_counts)])

# Customize the layout of the plot
fig.update_layout(
    title_text="Word Count Distribution",  # Set the title of the plot
    title_x=0.5,  # Center the title horizontally
    xaxis_title="Number of Words",  # Label for x-axis
    yaxis_title="Frequency",  # Label for y-axis
    bargap=0.1,  # Add some gap between bars for better readability
)

# Display the plot
fig.show()

In [ ]:
from collections import Counter
import plotly.graph_objects as go


def plot_histogram_word(text_list, n_most_common=30):
    # Create a list of all words from all texts
    words = [word for txt in text_list for word in txt.split()]

    # Count the frequency of each word
    word_counts = Counter(words)

    # Select the top n most frequent words
    top_words = dict(word_counts.most_common(n_most_common))

    # Create a bar chart using Plotly
    fig = go.Figure(
        [
            go.Bar(
                x=list(top_words.keys()),
                y=list(top_words.values()),
                text=list(top_words.values()),  # Display count on each bar
                textposition="auto",
            )
        ]
    )

    # Customize the layout
    fig.update_layout(
        title_text=f"Top {n_most_common} most frequent words in the text",
        title_x=0.5,  # Center the title
        xaxis_title="Words",
        yaxis_title="Frequency",
        xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
    )

    # Display the plot
    fig.show()


# Call the function with the cleaned training data
plot_histogram_word(denuncias_limpas, 30)

In [ ]:
### Retirando stops words
!python -m spacy download pt_core_news_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/pt_core_news_sm-3.8.0/pt_core_news_sm-3.8.0-py3-none-any.whl (13.0 MB)
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import nltk
from nltk.corpus import stopwords


nltk.download("stopwords")
stopwords_nltk = stopwords.words("portuguese")

list(stopwords_nltk)[:10]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['a',
 'à',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as']

In [ ]:
import spacy


nlp = spacy.load("pt_core_news_sm")
stopwords_spacy = nlp.Defaults.stop_words
list(stopwords_spacy)[:10]

['próximo',
 'meses',
 'teus',
 'bom',
 'direita',
 'estado',
 'sei',
 'tens',
 'deste',
 'onze']

In [ ]:
both_stopwords = set(stopwords_nltk) | set(stopwords_spacy)
len(both_stopwords)

def remove_stopwords(text):
    # Split the input text into individual tokens (words)
    tokens = text.lower().split()

    # Filter out tokens that are present in the both_stopwords set
    tokens = filter(lambda token: token.lower() not in both_stopwords, tokens)

    # Join the filtered tokens back into a single string with spaces in between
    return " ".join(tokens)


denuncias_limpas_sem_stopwords = denuncias_limpas.apply(remove_stopwords)

In [ ]:
plot_histogram_word(denuncias_limpas_sem_stopwords, 30)

In [ ]:
def spacy_lemmatizer(text):
    doc = nlp(text)
    txt = [token.lemma_ for token in doc]
    return " ".join(txt)

In [ ]:
denuncias_limpas_sem_stopwords_lemarizadas = denuncias_limpas.apply(spacy_lemmatizer)

In [ ]:
plot_histogram_word(denuncias_limpas_sem_stopwords_lemarizadas, 30)

In [ ]:
def deixa_apenas_palavras_3_caracteres(textos):
    return [word for word in textos if len(word) > 2]

Melhorando a pipeline de limpeza dos dados

In [ ]:
pipeline_clean_text = Pipeline(
    [
        ("apenas_caracteres_importantes", FunctionTransformer(apenas_caracteres_importantes)),
        ("spacy_lemmatizer", FunctionTransformer(spacy_lemmatizer)),
        ("remove_stopwords", FunctionTransformer(remove_stopwords))
    ]
)

denuncias_limpas_v2 = denuncias.apply(pipeline_clean_text.transform)
len(denuncias_limpas)

15415

In [ ]:
plot_histogram_word(denuncias_limpas_v2, 30)

In [ ]:
!python -m nltk.downloader punkt averaged_perceptron_tagger stopwords

<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import nltk

nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

from collections import Counter  # Import Counter to count occurrences of n-grams
from nltk.util import ngrams  # Import ngrams to generate n-grams from text
from nltk import word_tokenize  # Import word_tokenize to split text into words
import plotly.graph_objs as go  # Import Plotly for creating visualizations


# Define a function to generate n-grams from text
def generate_ngrams(text, n, lowercase=False):
    # Convert text to lowercase if specified
    if lowercase:
        text = text.lower()

    # Tokenize the text into words
    tokens = nltk.word_tokenize(text)

    # Generate n-grams from the tokenized words
    n_grams = ngrams(tokens, n)

    # Join the n-grams into strings and return as a list
    return [" ".join(grams) for grams in n_grams]


# Initialize a Counter to count occurrences of n-grams
n_grams_counter = Counter()

# Define the number of most common n-grams to display
n_most_common = 30

# Loop through each text in the preprocessed training dataset
for text in denuncias_limpas_v2.values:
    # Generate bigrams (n=2) from the text and update the counter
    n_grams_counter.update(generate_ngrams(text, n=2, lowercase=True))

# Select the top 30 most frequent bigrams
n_grams_counter = dict(n_grams_counter.most_common(n_most_common))

# Create a bar chart to visualize the most frequent bigrams
fig = go.Figure(
    [go.Bar(x=list(n_grams_counter.keys()), y=list(n_grams_counter.values()))]
)

# Update the layout of the figure with a title
fig.update_layout(title_text=f"Top {n_most_common} most frequent bigrams in the text")

# Display the figure
fig.show()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [ ]:
# Initialize a Counter to count occurrences of n-grams
n_grams_counter = Counter()

# Define the number of most common n-grams to display
n_most_common = 30

# Loop through each text in the preprocessed training dataset
for text in denuncias_limpas_v2.values:
    # Generate bigrams (n=2) from the text and update the counter
    n_grams_counter.update(generate_ngrams(text, n=3, lowercase=True))

# Select the top 30 most frequent bigrams
n_grams_counter = dict(n_grams_counter.most_common(n_most_common))

# Create a bar chart to visualize the most frequent bigrams
fig = go.Figure(
    [go.Bar(x=list(n_grams_counter.keys()), y=list(n_grams_counter.values()))]
)

# Update the layout of the figure with a title
fig.update_layout(title_text=f"Top {n_most_common} most frequent bigrams in the text")

# Display the figure
fig.show()